In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,confusion_matrix
import sklearn
from sklearn import pipeline  ,ensemble 
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.pipeline import Pipeline 
from sklearn import impute
from sklearn import compose
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, plot_confusion_matrix,roc_auc_score
from sklearn import set_config
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler,Normalizer,PowerTransformer,QuantileTransformer, RobustScaler,StandardScaler,LabelEncoder, OneHotEncoder,OrdinalEncoder
from sklearn.tree          import DecisionTreeClassifier
from sklearn.linear_model  import LogisticRegression    
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
# from catboost              import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
set_config(display='diagram') # Useful for display the pipeline
print("Pandas  ", pd.__version__)
print("Sklearn ", sklearn.__version__) # Try to use 0.24



Pandas   1.2.4
Sklearn  0.24.2


In [2]:
data = pd.read_csv('./data/data_5secondWindow .csv')
data.head(5)

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,Luca
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,Luca
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,andrea


In [3]:
def change_name(df):
    column_names=[]
    for i in df.columns:
        k=i.replace('.','_').replace("#",'_')
        df.rename(columns = {i:k}, inplace = True)
    return df
data=change_name(data)

# Create test data 

In [4]:
# create test data from some user 
df=data.copy()
user1=df[df['user']=='Pierpaolo']
user2=df[df['user']=='IvanHeibi']
user3=df[df['user']=='AndreaCarpineti']
user4=df[df['user']=='Elena']
users=[user1,user2,user3,user4]
df_test=pd.concat(users)
df_test.head(5)

,id,time,activityrecognition_0,activityrecognition_1,android_sensor_accelerometer_mean,android_sensor_accelerometer_min,android_sensor_accelerometer_max,android_sensor_accelerometer_std,android_sensor_game_rotation_vector_mean,android_sensor_game_rotation_vector_min,...,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
33,19919,260.0,NaN,5.0,11.068949,9.764513,14.028126,1.283204,0.943836,0.936799,...,77.064227,77.064227,77.064227,4.549349,1.191050,1.191050,1.191050,NaN,Walking,Pierpaolo
54,19494,78.0,NaN,75.0,9.263056,8.712177,9.963264,0.361961,0.894102,0.894102,...,74.178611,74.178611,74.178611,0.149748,0.000000,0.000000,0.000000,NaN,Bus,Pierpaolo
102,20104,445.0,NaN,5.0,9.075213,9.012243,9.187340,0.070605,0.907671,0.905519,...,6.020600,6.020600,6.020600,0.000000,0.000000,0.000000,0.000000,0.0,Walking,Pierpaolo
121,19436,20.0,NaN,100.0,9.061146,8.246220,9.563967,0.451773,0.997597,0.997312,...,72.559600,72.559600,72.559600,NaN,7.800006,7.800006,7.800006,NaN,Bus,Pierpaolo
172,20245,6.0,NaN,100.0,10.335784,3.332163,17.743662,4.272022,0.991409,0.988798,...,73.702289,73.702289,73.702289,NaN,1.250000,1.250000,1.250000,NaN,Walking,Pierpaolo


## Subtract test data from data 

In [5]:
df_train = df.merge(df_test, how='left', indicator=True)
df_train = df_train[df_train['_merge'] == 'left_only']
df_train=df_train.drop('_merge',axis=1)
df_train.head(5)

,id,time,activityrecognition_0,activityrecognition_1,android_sensor_accelerometer_mean,android_sensor_accelerometer_min,android_sensor_accelerometer_max,android_sensor_accelerometer_std,android_sensor_game_rotation_vector_mean,android_sensor_game_rotation_vector_min,...,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,Luca
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,Luca
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,andrea


In [6]:

train_missing = (df_train.isnull().sum() / len(df_train)).sort_values(ascending = False)
test_missing = (df_test.isnull().sum() / len(df_test)).sort_values(ascending = False)

train_missing = train_missing.index[train_missing > 0.75]
test_missing = test_missing.index[test_missing > 0.75]

all_missing = list(set(set(train_missing) | set(test_missing)))
print('There are %d columns with more than 75%% missing values' % len(all_missing))

There are 11 columns with more than 75% missing values


In [7]:
# Drop missing data 
df_train = df_train.drop(all_missing,axis=1)
df_test  = df_test.drop(all_missing,axis=1)

In [9]:
print(df_train.shape)
print(df_test.shape)

(4985, 56)
(908, 56)


## drop id ,time, user columns

In [8]:
unimp    = ['id','time','user']
df_train = df_train.drop(unimp,axis=1)
df_test  = df_test.drop(unimp,axis=1)

In [10]:
print(df_train.shape)
print(df_test.shape)

(4985, 56)
(908, 56)


In [11]:
X=df_train.drop('target',axis=1)
y=df_train.target
X_test=df_test.drop('target',axis=1)
y_test=df_test.target

# Pipeline 

In [12]:
num_attribs=df_train.drop('target',axis=1).columns.to_list()
num_pip=Pipeline([('imputer',impute.SimpleImputer(strategy='median')),
                  ('scalar',MinMaxScaler()),
                 ('PCA',PCA(n_components=12))])
preprocessor=compose.ColumnTransformer([
    ('num',num_pip,num_attribs)
])

In [13]:
# pipeline for model
classifier_models = {
"DecisionTreeClassifier": DecisionTreeClassifier(),
'KNeighborsClassifier':KNeighborsClassifier(),
'LogisticRegression':LogisticRegression(),
"RandomForestClassifier":RandomForestClassifier(),
"GradientBoostingClassifier":GradientBoostingClassifier(),
"XGBClassifier":XGBClassifier(),
"LGBMClassifier":LGBMClassifier(),
}
# make pipline with  preprocessing 
classifier_models = {name: pipeline.make_pipeline(preprocessor, model) for name, model in classifier_models.items()}
classifier_models["GradientBoostingClassifier"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scalar',
                                                                   MinMaxScaler()),
                                                                  ('PCA',
                                                                   PCA(n_components=12))]),
                                                  ['activityrecognition_1',
                                                   'android_sensor_accelerometer_mean',
                                                   'android_sensor_accelerometer_min',
                                                   'android_sensor_accelerometer_max',
                                                   'android_sensor_accelerom...
                                                   'android_sensor_light_min',
                                                   'android_sensor_light_max',
                                                   'android_sensor_light_std',
                                                   'android_sensor_linear_acceleration_mean',
                                                   'android_sensor_linear_acceleration_min',
                                                   'android_sensor_linear_acceleration_max',
                                                   'android_sensor_linear_acceleration_std',
                                                   'android_sensor_magnetic_field_mean', ...])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

In [28]:


x_train, x_val, y_train, y_val = train_test_split(X, y,test_size=0.4,stratify = y,random_state=10 )
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [],'Time': []})
for model_name, model in classifier_models.items():
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    valid_pred = model.predict(x_val)
    results = results.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_val, valid_pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y_val, valid_pred)*100,
                    
                              "Time":     total_time},
                              ignore_index=True)
results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


[05:10:36] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Accuracy,Bal Acc.,Time
1,LGBMClassifier,96.138415,95.746435,0.997976
2,XGBClassifier,95.887663,95.616522,2.065293
3,RandomForestClassifier,95.687061,95.344613,0.880386
4,KNeighborsClassifier,94.433300,94.408352,0.069052
5,GradientBoostingClassifier,93.831494,93.489355,5.550231
6,DecisionTreeClassifier,90.120361,89.891901,0.122440
7,LogisticRegression,73.871615,74.937257,0.199365


# Best model for test data  

In [16]:
from sklearn.calibration import CalibratedClassifierCV

In [32]:
best_model=CalibratedClassifierCV(base_estimator=XGBClassifier())
model_name='CalibratedClassifierCV'

In [33]:
clf=Pipeline([('preprocess',preprocessor),('classification',best_model)])

In [38]:

result = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [],'Time': []})
for model_name, model in classifier_models.items():
    start_time = time.time()
    model.fit(X, y)
    total_time = time.time() - start_time
    pred = model.predict(X_test)
    result = result.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_test, pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
result_ord = result.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
result_ord.index += 1 
result_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


[05:15:30] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Accuracy,Bal Acc.,Time
1,GradientBoostingClassifier,44.713656,49.187624,9.560127
2,XGBClassifier,44.713656,51.495509,3.401669
3,LGBMClassifier,39.977974,49.591188,1.107100
4,KNeighborsClassifier,38.986784,41.854042,0.132283
5,RandomForestClassifier,37.444934,44.749290,1.386523
6,LogisticRegression,36.013216,51.980448,0.545893
7,DecisionTreeClassifier,31.718062,28.765215,0.249218


In [39]:
print(classification_report(y_test, pred))
print()
print(confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

         Bus       0.29      0.23      0.26       164
         Car       0.08      0.59      0.14        34
       Still       0.42      0.80      0.55        76
       Train       0.28      0.49      0.35        86
     Walking       0.91      0.37      0.53       548

    accuracy                           0.40       908
   macro avg       0.40      0.50      0.37       908
weighted avg       0.67      0.40      0.45       908


[[ 38  41  41  26  18]
 [  0  20   1  13   0]
 [  1   8  61   5   1]
 [ 28  16   0  42   0]
 [ 63 177  41  65 202]]


# tuning model 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:

k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)             

In [ ]:
# instantiate the grid
grid = GridSearchCV(clf, param_grid, cv=2, scoring='accuracy')

In [ ]:
>>> from sklearn.model_selection import GridSearchCV
 from sklearn.calibration import CalibratedClassifierCV
>>> from sklearn.ensemble import RandomForestClassifier
>>> from sklearn.datasets import make_moons
>>> X, y = make_moons()
>>> calibrated_forest = CalibratedClassifierCV(
...    base_estimator=RandomForestClassifier(n_estimators=10))